# ACADEMIC HISTORY PARSER

## NOTE - 
The academic history should be in ***docx*** file only

In [ ]:
import docx2txt
import pandas as pd
import numpy as np

In [ ]:
class Student_details:
    regNo = ""
    name = ""
    program = ""
    school = ""
    creditsRegistered = 0
    creditsEarned = 0
    CGPA = 0
    
    def getNameDetails(self, metadata, value):
        if(metadata == "Reg. No."):
            self.regNo = value
        if(metadata == "Name"):
            self.name = value.title()
        if(metadata == "Program"):
            self.program = value[8:]
        if(metadata == "School"):
            self.school = value

    keys = ['S', 'A', 'B', 'C', 'D', 'E', 'F', 'N']
    gradeCategories = dict.fromkeys(keys, 0)
    noOfCreditsRegistered = 0
    noOfCreditsAcquired = 0
    
    def getGradeCategories(self, value):
        for i in range(0, 8):
            self.gradeCategories[self.keys[i]] = value[-(8-i)]
        self.noOfCreditsRegistered = int(value[3])
        self.noOfCreditsAcquired = int(value[4])
        self.CGPA = float(value[5])

In [ ]:
#Instance of the student
student = Student_details()

In [ ]:
#convert DOC -> STRING, split STRING -> LIST CONTAINING LINES
abc = docx2txt.process(r'history.docx')
content = []
for line in abc.splitlines():
  if line != '':
    content.append(line)

In [ ]:
#Get STUDENT details
metadata = content[0:11]
metadata_length = len(metadata)

for i in range(metadata_length):
    if(i < metadata_length-1):
        student.getNameDetails(metadata[i], metadata[i+1])
    
print("Name -> ", student.name)
print("Reg. No. -> ", student.regNo)
print("Program ->", student.program)
print("School ->", student.school)

In [ ]:
divider = content.index('Credits Registered')
subjects_data = content[21:divider]
subjects_data_length = len(subjects_data)

In [ ]:
subjects_list = []
counter = 2
a = []

for i in range(subjects_data_length):
    if(i < subjects_data_length-1):
        if(subjects_data[i] == str(counter) and len(subjects_data[i+1]) == 7):
            subjects_list.append(a)
            counter+=1
            a = []
    a.append(subjects_data[i])

subjects_list.append(a)

In [ ]:
#Get CREDIT details
credit_data = content[divider:]
student.getGradeCategories(credit_data)
print(student.gradeCategories)
print("No. of Credits Registered -> ", student.noOfCreditsRegistered)
print("No. of Credits Acquired -> ", student.noOfCreditsAcquired)
print("CGPA ->", student.CGPA)

In [ ]:
#Assign data to dataframe, and filter out the meaningful stuff alone
column_labels = ["S.No.", "Course_Code", 
                 "Course Name", "Course Type", 
                 "Credit", "Grade", "Exam Held", 
                 "Result Date", "Course Option"]
p = pd.DataFrame(subjects_list, columns = column_labels)
academic_dataframe = p[p.Credit != '-']

In [ ]:
#Rectify S.No. continuity and rectify index labels
ah_size = academic_dataframe['Course_Code'].count()
s_no = np.arange(1, ah_size+1)
academic_dataframe['S.No.'] = s_no.transpose()
academic_dataframe = academic_dataframe.set_index('S.No.', drop=True)

## NOTE - 
The index of the rows of the dataframe is from 1 to n, not 0 to n-1

In [ ]:
# FOR REF - fetch the first value
academic_dataframe.loc[1,'Course_Code']

In [ ]:
#Change Credits column from character to integers
academic_dataframe['Credit'] = pd.to_numeric(academic_dataframe['Credit'])

In [ ]:
academic_dataframe

In [ ]:
#Calculate the no. of credits completed excluding N and F grades
exclude_failed_courses = academic_dataframe[academic_dataframe['Grade'] != ('N' or 'F')]
total_credits = np.sum(exclude_failed_courses['Credit'])
print("Credits completed till date -> ", total_credits)